In [3]:
# import os
# from ftplib import FTP

# def grabFile(filename): # FTP download and save function
#     # if the file already exists, skip it
#     if os.path.isfile(local_dir+'/'+filename):
#         return
    
#     localfile = open(local_dir+'/'+filename, 'wb') # local download
#     ftp.retrbinary('RETR ' + filename, localfile.write, 1024) # FTP saver

#     localfile.close() # close local file
    
# ftp = FTP('ftp.avl.class.noaa.gov')  # connect to NOAA's FTP
# ftp.login() # login anonymously
# ftp.cwd('./8152909373') # navigate to exact FTP folder where data is located

# files = ftp.nlst() # collect file names into vector

# print(files)

# # Creates a local GOES repository in the current directory
# local_dir = '/Users/jen/GitHub/GOES_files'
# if os.path.isdir(local_dir)==False:
#     os.mkdir(local_dir)

# # loop through files and save them one-by-one
# for filename in files:
#     grabFile(filename)

# ftp.quit() # close FTP connection

In [1]:
import os
from netCDF4 import Dataset # netCDF library

local_dir = 'data/GOES_files'
if os.path.isdir(local_dir):
    files = os.listdir(local_dir)
else:
    print('No GOES files in directory')

file = Dataset(local_dir+'/'+files[0]) # open local netCDF GOES file

In [11]:
local_dir

'data/GOES_files'

In [2]:
file.summary

"The Land Surface (Skin) Temperature product consists of pixels containing the skin temperatures for each 'clear' or 'probably clear' land surface pixel. This product is generated from a regression algorithm that linearly combines ABI surface emissivity data, brightness temperature, and brightness temperature differences derived from top of atmosphere radiances from ABI bands with wavelengths 11.19 and 12.27 um. Product data is generated both day and night."

In [3]:
for ii in file.variables:
    print(ii)

LST
DQF
PQI
t
y
x
time_bounds
goes_imager_projection
y_image
y_image_bounds
x_image
x_image_bounds
nominal_satellite_subpoint_lat
nominal_satellite_subpoint_lon
nominal_satellite_height
geospatial_lat_lon_extent
retrieval_local_zenith_angle
quantitative_local_zenith_angle
retrieval_local_zenith_angle_bounds
quantitative_local_zenith_angle_bounds
solar_zenith_angle
solar_zenith_angle_bounds
total_pixels_where_lst_is_retrieved
number_good_retrievals
outlier_pixel_count
min_lst
max_lst
mean_lst
standard_deviation_lst
FPT_mitigation_flag
algorithm_dynamic_input_data_container
processing_parm_version_container
algorithm_product_version_container
percent_uncorrectable_GRB_errors
percent_uncorrectable_L0_errors


In [4]:
file.variables['LST'].units

'K'

In [8]:
# conda install basemap

In [9]:
# conda install -c conda-forge basemap-data-hires

In [6]:
from netCDF4 import Dataset
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np

bbox = [-124.763068,24.523096,-66.949895,49.384358] # map boundaries
# figure setup
fig,ax = plt.subplots(figsize=(9,4),dpi=200)
ax.set_axis_off()
# set basemap boundaries, cylindrical projection, 'i' = intermediate resolution
m = Basemap(llcrnrlon=bbox[0],llcrnrlat=bbox[1],urcrnrlon=bbox[2],
            urcrnrlat=bbox[3],resolution='i', projection='cyl')

m.fillcontinents(color='#d9b38c',lake_color='#bdd5d5') # continent colors
m.drawmapboundary(fill_color='#bdd5d5') # ocean color
m.drawcoastlines() 
m.drawcountries()
states = m.drawstates() # draw state boundaries
# parallels = np.linspace(bbox[0],bbox[2],5.) # longitude lines
# m.drawparallels(parallels,labels=[True,False,False,False])
# meridians = np.linspace(bbox[1],bbox[3],5.)  # latitude lines
# m.drawmeridians(meridians,labels=[False,False,False,True])

plt.show()

In [12]:
from netCDF4 import Dataset
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import numpy as np
import os

def lat_lon_reproj(nc_folder):
    os.chdir(nc_folder)
    full_direc = os.listdir()
    nc_files = [ii for ii in full_direc if ii.endswith('.nc')]
    g16_data_file = nc_files[0] # select .nc file
    print(nc_files[0]) # print file name

    # designate dataset
    g16nc = Dataset(g16_data_file, 'r')
    var_names = [ii for ii in g16nc.variables]
    var_name = var_names[0]

    # GOES-R projection info and retrieving relevant constants
    proj_info = g16nc.variables['goes_imager_projection']
    lon_origin = proj_info.longitude_of_projection_origin
    H = proj_info.perspective_point_height+proj_info.semi_major_axis
    r_eq = proj_info.semi_major_axis
    r_pol = proj_info.semi_minor_axis

    # grid info
    lat_rad_1d = g16nc.variables['x'][:]
    lon_rad_1d = g16nc.variables['y'][:]
    
    # close file when finished
    g16nc.close()
    g16nc = None

    # create meshgrid filled with radian angles
    lat_rad,lon_rad = np.meshgrid(lat_rad_1d,lon_rad_1d)

    # lat/lon calc routine from satellite radian angle vectors

    lambda_0 = (lon_origin*np.pi)/180.0

    a_var = np.power(np.sin(lat_rad),2.0) + (np.power(np.cos(lat_rad),2.0)*(np.power(np.cos(lon_rad),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(lon_rad),2.0))))
    b_var = -2.0*H*np.cos(lat_rad)*np.cos(lon_rad)
    c_var = (H**2.0)-(r_eq**2.0)

    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)

    s_x = r_s*np.cos(lat_rad)*np.cos(lon_rad)
    s_y = - r_s*np.sin(lat_rad)
    s_z = r_s*np.cos(lat_rad)*np.sin(lon_rad)

    # latitude and longitude projection for plotting data on traditional lat/lon maps
    lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)

#     os.chdir('../')

    return lon,lat

def data_grab(nc_folder,nc_indx):
    os.chdir(nc_folder)
    full_direc = os.listdir()
    nc_files = [ii for ii in full_direc if ii.endswith('.nc')]
    g16_data_file = nc_files[nc_indx] # select .nc file
    print(nc_files[nc_indx]) # print file name

    # designate dataset
    g16nc = Dataset(g16_data_file, 'r')
    var_names = [ii for ii in g16nc.variables]
    var_name = var_names[0]

    # data info    
    data = g16nc.variables[var_name][:]
    data_units = g16nc.variables[var_name].units
    data_time_grab = ((g16nc.time_coverage_end).replace('T',' ')).replace('Z','')
    data_long_name = g16nc.variables[var_name].long_name
    
    # close file when finished
    g16nc.close()
    g16nc = None

#     os.chdir('../')
    # print test coordinates
    print('{} N, {} W'.format(lat[318,1849],abs(lon[318,1849])))

    return data,data_units,data_time_grab,data_long_name,var_name

nc_folder = 'data/GOES_files' # define folder where .nc files are located
lon,lat = lat_lon_reproj(nc_folder)

file_indx = 10 # be sure to pick the correct file. Make sure the file is not too big either,
# some of the bands create large files (stick to band 7-16)

data,data_units,data_time_grab,data_long_name,var_name = data_grab('data/GOES_files',file_indx)
# main data grab from function above

data_bounds = np.where(data.data!=65535)
bbox = [np.min(lon[data_bounds]),
        np.min(lat[data_bounds]),
        np.max(lon[data_bounds]),
        np.max(lat[data_bounds])] # set bounds for plotting

# figure routine for visualization
fig = plt.figure(figsize=(9,4),dpi=200)

n_add = 0 # for zooming in and out
m = Basemap(llcrnrlon=bbox[0]-n_add,llcrnrlat=bbox[1]-n_add,urcrnrlon=bbox[2]+n_add,urcrnrlat=bbox[3]+n_add,resolution='i', projection='cyl')
m.fillcontinents(color='#d9b38c',lake_color='#bdd5d5',zorder=1) # continent colors
m.drawmapboundary(fill_color='#bdd5d5',zorder=0) # ocean color
m.drawcoastlines(linewidth=0.5)
m.drawcountries(linewidth=0.25)
m.drawstates(zorder=2)
m.pcolormesh(lon.data, lat.data, data, latlon=True,zorder=999) # plotting actual LST data
# parallels = np.linspace(bbox[1],bbox[3],5.)
# m.drawparallels(parallels,labels=[True,False,False,False],zorder=2,fontsize=8)
# meridians = np.linspace(bbox[0],bbox[2],5.)
# m.drawmeridians(meridians,labels=[False,False,False,True],zorder=1,fontsize=8)
cb = m.colorbar()

data_units = ((data_units.replace('-','^{-')).replace('1','1}')).replace('2','2}')
plt.rc('text', usetex=True)
cb.set_label(r'%s $ \left[ \mathrm \right] $'% (var_name,data_units))
plt.title(' on '.format(data_long_name,data_time_grab))
plt.tight_layout()

plt.savefig('goes_16_data_demo.png',dpi=200,facecolor=[252/255,252/255,252/255]) # uncomment to save figure
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'data/GOES_files'